In [6]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %code_wrap  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %mamba  %man  %matplotlib  %micromamba  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%code_wrap  %%debug  %%file  %%html  %%javascript  %%js  %

In [7]:
#-------------------------------------------------------------------------------------------------------------------------------
from pyspark import SparkContext                                    # Importing SparkContext
from pyspark.sql import SparkSession, Window, Row                   # Importing importing methods for creating a cluster
from pyspark.sql import functions as F                              # Importing SQL Functions
from pyspark.sql.functions import col, isnan, when, count           # Importing relevant dataframe functions
from pyspark.sql.functions import *                                 # Importing inbuilt SQL Functions
from pyspark.sql.types import *                                     # Importing SQL types
#-------------------------------------------------------------------------------------------------------------------------------
import matplotlib.pyplot as plt                                     # Popular plotting library
%matplotlib inline
import seaborn as sns                                               # Advanced plotting library
from handyspark import *                                            # Helper library to plot graphs
#-------------------------------------------------------------------------------------------------------------------------------
from pyspark.ml.feature import VectorAssembler                      # For processing dataset for ML
#-------------------------------------------------------------------------------------------------------------------------------
import warnings                                                     # Importing warning to disable runtime warnings
warnings.filterwarnings("ignore")                                   # Warnings will appear only once
#-------------------------------------------------------------------------------------------------------------------------------

/home/team/.local/share/virtualenvs/Big-Data-Analytics-ofTsS-TY/lib/python3.11/site-packages/handyspark/sql/transform.py:9: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  'boolean': np.bool,


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
# Building a spark app/session
spark = SparkSession.builder.appName("carsSpark").getOrCreate()

# single cluster information
spark

In [ ]:
# Reading the data
df = spark.read.csv('SalesAnalysis.csv', header=True, inferSchema="true")

# Shape of the dataset
print('Shape of the dataset: ', (df.count(), len(df.columns)))

# Displaying top n=10 rows
df.show(n=10)

In [ ]:
# Alternatively, displaying the head of the data
df.head(n=5)

In [ ]:
# Getting a description of the dataset, similar to pandas df.describe()
df.describe().show()

In [ ]:
# Checking for dataframe schema, similar to pandas df.info()
df.printSchema()   

In [ ]:
# Displaying the values of a particular column
df.select('Selling_Price').show()

In [ ]:
# Selecting multiple columns
df.select(['Selling_Price','Present_Price']).show()

In [ ]:
# Checking type of dataframe
print(type(df))

# Checking type of column data
print(type(df.select('Selling_Price')))

In [ ]:
# Displaying dtypes of columns
df.dtypes

In [ ]:
df1 = df.limit(5)
df1.show()

In [ ]:
# getting the list of Row objects
row_list = df.collect()
  
# Slicing the Python List
part1 = row_list[:5]
part2 = row_list[5:]
  
# Converting the slices to PySpark DataFrames
slice1 = spark.createDataFrame(part1)
slice2 = spark.createDataFrame(part2)
  
# Printing the first slice
print('First DataFrame')
slice1.show()
  
# Printing the second slice
print('Second DataFrame')
slice2.show()

In [ ]:
# Adding a new column to the dataset
df1 = df.withColumn("Car New", df['Present_Price']*2)

# Displaying the new dataframe
df1.show()

In [ ]:
# Dropping the newly added column
df2 = df1.drop('Car New')

# Displaying the new dataframe
df2.show()

In [ ]:
data = df.toPandas()
print('Type of dataframe:', type(data))
print('Shape of the dataframe (Pandas version): ', data.shape)
data.head()

In [ ]:
# Creating a dataframe to check null value counts
null_df = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '') | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])

# Displaying the null value counts dataframe
null_df.show()

In [ ]:
# Converting Year to datetime
df1 = df.withColumn("Year", F.to_date(F.col("Year").cast("string"), 'yyyy'))

# Extracting Year
df = df1.withColumn('Year', F.year(F.to_timestamp('Year', 'yyyy')))
df.show()

In [ ]:
# Converting to handyspark dataframe
hdf = df.toHandy()
hdf.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
hdf.cols['Selling_Price'].hist(ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
hdf.cols['Seller_Type'].hist(ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
hdf.cols['Transmission'].hist(ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
hdf.cols[['Kms_Driven', 'Selling_Price']].scatterplot(ax=ax)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
hdf.cols[['Selling_Price', 'Present_Price']].scatterplot(ax=ax)
plt.show()

In [ ]:
hdf.stratify(['Transmission']).cols['Selling_Price'].mean()

In [ ]:
hdf.stratify(['Transmission']).cols['Selling_Price'].hist(figsize=(15,7))
plt.show()

In [ ]:
hdf.stratify(['Seller_Type']).cols['Selling_Price'].hist(figsize=(15,7))
plt.show()

In [ ]:
hdf.stratify(['Fuel_Type']).cols['Selling_Price'].hist(figsize=(15,7))
plt.show()